# 03 — Modeling (Logistic Regression & Random Forest)

本 Notebook 使用整理後的 train dataset 建立分類模型，並進行評估、比較、解釋特徵影響。

---

## 📌 目標

- 建立 baseline（Logistic Regression）
- 建立非線性模型（Random Forest）
- 比較兩者效能差異
- 解釋模型結果（係數、重要性）
- 判讀 Type I / Type II error

---

## 📘 目錄

### 1. 匯入清洗後的資料
- 匯入 cleaned_train
- 分離 feature X / target y

### 2. 切分訓練與驗證集
- train_test_split
- 設定 random_state 以確保重現性

### 3. 建立 Logistic Regression baseline
- 訓練模型
- Accuracy / Precision / Recall / F1-score
- Confusion Matrix
- 係數（Coefficients）解析

### 4. 建立 Random Forest 模型
- 訓練模型
- Hyperparameter（如 max_depth, n_estimators）
- Validation accuracy
- Confusion Matrix
- Feature Importance 排名

### 5. 模型比較與解釋
- Logistic vs. Random Forest（各自優勢）
- 哪些特徵提升預測表現？
- 業務意義：哪些特徵代表重要客戶行為？

---

## 🎯 最終成果

一份完整的模型評估報告，包含：
- 最佳模型選擇（通常是 RF）
- 特徵重要性（可用於商業洞察）
- 完整混淆矩陣分析（利害關係）

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 從 02 輸出的清洗後資料讀回來
train = pd.read_csv("train_clean.csv")

print(train.head())
print(train.dtypes)

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Sex  Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    0   22      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1   38      1      0   
2                             Heikkinen, Miss. Laina    1   26      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1   35      1      0   
4                           Allen, Mr. William Henry    0   35      0      0   

             Ticket     Fare  FamilySize  Embarked_Q  Embarked_S  
0         A/5 21171   7.2500           2       False        True  
1          PC 17599  71.2833           2       False       False  
2  STON/O2. 3101282   7.9250           1       False        True  
3            113803  53.1000      

In [2]:
# 2. 指定要用來建模的特徵欄位
features = [
    "Pclass",
    "Sex",
    "Age",
    "SibSp",
    "Parch",
    "Fare",
    "FamilySize",
    "Embarked_Q",
    "Embarked_S",
]

X = train[features]          # 特徵
y = train["Survived"]        # 目標變數

# 確認沒有缺失值（保險檢查）
print("X 缺失值：\n", X.isnull().sum())
print("y 缺失值：\n", y.isnull().sum())

X 缺失值：
 Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
FamilySize    0
Embarked_Q    0
Embarked_S    0
dtype: int64
y 缺失值：
 0


In [3]:
# 3. 切分訓練集 / 驗證集
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,        # 20% 當驗證集
    random_state=42,      # 固定亂數，讓結果可重現
    stratify=y            # 依 Survived 分層抽樣 → 生存/死亡比例一致
)

X_train.shape, X_val.shape

((712, 9), (179, 9))

In [4]:
# 4. 建立 Logistic Regression baseline 模型
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [5]:
# 5. 在驗證集上評估 Logistic Regression
y_pred = model.predict(X_val)

print("=== Logistic Regression 評估結果 ===")
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))

=== Logistic Regression 評估結果 ===
Accuracy: 0.8044692737430168
Confusion Matrix:
 [[98 12]
 [23 46]]
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.89      0.85       110
           1       0.79      0.67      0.72        69

    accuracy                           0.80       179
   macro avg       0.80      0.78      0.79       179
weighted avg       0.80      0.80      0.80       179



In [6]:
# 6. 查看 Logistic Regression 的特徵係數（解釋用）
print("Logistic Regression 的特徵係數：\n")

coefficients = pd.DataFrame({
    "Feature": X_train.columns,
    "Coefficient": model.coef_[0]
}).sort_values(by="Coefficient", ascending=False)

coefficients

Logistic Regression 的特徵係數：



,Feature,Coefficient
1,Sex,2.559862
7,Embarked_Q,0.285640
4,Parch,0.043930
5,Fare,0.002253
2,Age,-0.038646
6,FamilySize,-0.116124
3,SibSp,-0.130273
8,Embarked_S,-0.376884
0,Pclass,-1.092823


In [7]:
# 7. 建立 Random Forest 模型（提升準確度）
rf = RandomForestClassifier(
    n_estimators=400,        # 樹的數量
    max_depth=None,         # 讓模型自己決定深度
    random_state=42,        # 固定亂數
    class_weight="balanced" # 處理正負樣本不平衡問題
)

rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=400,
                       random_state=42)

In [8]:
# 8. 在驗證集上評估 Random Forest
y_pred_rf = rf.predict(X_val)

print("=== Random Forest 評估結果 ===")
print("Accuracy:", accuracy_score(y_val, y_pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_rf))
print("Classification Report:\n", classification_report(y_val, y_pred_rf))

=== Random Forest 評估結果 ===
Accuracy: 0.8268156424581006
Confusion Matrix:
 [[97 13]
 [18 51]]
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.88      0.86       110
           1       0.80      0.74      0.77        69

    accuracy                           0.83       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.83      0.83      0.83       179



In [9]:
# 9. 查看 Random Forest 的特徵重要性
rf_importance = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": rf.feature_importances_
}).sort_values(by="Importance", ascending=False)

rf_importance

,Feature,Importance
5,Fare,0.272109
1,Sex,0.259662
2,Age,0.243472
0,Pclass,0.080380
6,FamilySize,0.053882
3,SibSp,0.029947
8,Embarked_S,0.025506
4,Parch,0.025501
7,Embarked_Q,0.009542
